# Car dashcam - Image Processing

## installing dependencies

In [1]:
! pip install opencv-python numpy scikit-image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 20.6 MB/s eta 0:00:00


mount google drive to get images and save the output result video

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


init variables that are used for input/output

In [3]:
input_video_path = '/content/drive/MyDrive/Colab Notebooks/Image-Processing-Project/video-dashcam-night.mp4'
output_video_path = '/content/drive/MyDrive/Colab Notebooks/Image-Processing-Project/highlighted_video-dashcam-night.mp4'

import cv2 as cv
import numpy as np

print(f"input video path: {input_video_path}")
print(f"output video path: {output_video_path}")

input video path: /content/drive/MyDrive/Colab Notebooks/Image-Processing-Project/video-dashcam-night.mp4
output video path: /content/drive/MyDrive/Colab Notebooks/Image-Processing-Project/highlighted_video-dashcam-night.mp4


## import video

In [4]:
cap = cv.VideoCapture(input_video_path)

if not cap.isOpened():
    print(f"eroare vere, nu merge deschis: {input_video_path}")
    exit()

frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv.CAP_PROP_FPS))

fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

if not out.isOpened():
    print(f"eroare vere, nu merge deschis video writer pt: {output_video_path}")

    cap.release()
    exit()

print(f"video capture: {input_video_path}")
print(f"video writer: {output_video_path}")
print(f"frame size: {frame_width}x{frame_height}, FPS: {fps}, Codec: XVID")


video capture: /content/drive/MyDrive/Colab Notebooks/Image-Processing-Project/video-dashcam-night.mp4
video writer: /content/drive/MyDrive/Colab Notebooks/Image-Processing-Project/highlighted_video-dashcam-night.mp4
frame size: 1920x1080, FPS: 30, Codec: XVID


function to process each frame and loop that iterates through each frame from the video

In [5]:
import cv2 as cv
import numpy as np

def process_frame(frame):
    # helper func to average and extrapolate lines

    def average_slope_intercept(image, lines):
        left_fit = []
        right_fit = []
        if lines is None:
            return None

        for line in lines:
            for x1, y1, x2, y2 in line:

                # to avoid division by zero for vertical lines with slope close to infinity
                if (x2 - x1) == 0:
                    continue

                fit = np.polyfit((x1, x2), (y1, y2), 1)
                slope = fit[0]
                intercept = fit[1]

                if slope < -0.1 and x1 < image.shape[1] / 2:  # left lane (negative slope and on left half)
                    left_fit.append((slope, intercept))
                elif slope > 0.1 and x1 > image.shape[1] / 2: # right lane (positive slope and on right half)
                    right_fit.append((slope, intercept))

        left_fit_average = np.average(left_fit, axis=0) if left_fit else None
        right_fit_average = np.average(right_fit, axis=0) if right_fit else None

        # func to make coordinates from slope and intercept
        def make_coordinates(image, line_params):
            if line_params is None:
                return None
            slope, intercept = line_params
            y1 = image.shape[0]  # bottom of the image
            y2 = int(y1 * (3/5)) # 3/5 of the image from bottom, or adjusted to top of ROI

            # ensure valid slope to prevent division by zero or extremely large values
            if abs(slope) < 0.001: # consider very small slopes as horizontal
                return None

            x1 = int((y1 - intercept) / slope)
            x2 = int((y2 - intercept) / slope)

            # make sure coordinates are within image bounds
            x1 = np.clip(x1, 0, image.shape[1])
            x2 = np.clip(x2, 0, image.shape[1])

            return np.array([x1, y1, x2, y2])

        left_line = make_coordinates(image, left_fit_average)
        right_line = make_coordinates(image, right_fit_average)

        return left_line, right_line

    # func to draw the lines on a empty image
    def draw_lines(img, lines, color=[0, 255, 0], thickness=10):
        line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
        if lines is not None:
            for line in lines:
                if line is not None:
                    x1, y1, x2, y2 = line
                    cv.line(line_img, (x1, y1), (x2, y2), color, thickness)
        return line_img


    # convert the frame from BGR to HSV color space
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    # lower and upper bounds for white color in HSV
    # basically the lower and upper limit for the road lines
    # lower limit means "linii sterse" are considered

    lower_white = np.array([0, 0, 120])
    upper_white = np.array([255, 30, 255])
    white_mask = cv.inRange(hsv, lower_white, upper_white)

    # lower and upper bounds for yellow color in HSV
    # same thing as for white
    lower_yellow = np.array([15, 100, 100])
    upper_yellow = np.array([35, 255, 255])
    yellow_mask = cv.inRange(hsv, lower_yellow, upper_yellow)

    # combine the white and yellow masks
    combined_mask = cv.bitwise_or(white_mask, yellow_mask)

    # apply the combined mask to the original frame
    masked_image = cv.bitwise_and(frame, frame, mask=combined_mask)

    # convert the masked image to grayscale
    gray_image = cv.cvtColor(masked_image, cv.COLOR_BGR2GRAY)

    # apply Gaussian blur to smooth the image and reduce noise
    blur_image = cv.GaussianBlur(gray_image, (5, 5), 0)

    # perform Canny edge detection
    canny_image = cv.Canny(blur_image, 50, 150)

    # define a region of interest (ROI)
    height, width = canny_image.shape[:2]
    roi_vertices = [
        (0, height),
        (width / 2, height / 2 + 50),
        (width, height)
    ]

    # create a blank mask image
    mask = np.zeros_like(canny_image)

    # fill the polygon with white color to create the ROI mask
    cv.fillPoly(mask, np.int32([roi_vertices]), 205)

    # apply the ROI mask to the Canny edge image
    roi_canny_image = cv.bitwise_and(canny_image, mask)

    # apply hough transform to detect line segments
    lines = cv.HoughLinesP(
        roi_canny_image,
        rho=2,
        theta=np.pi / 180,
        threshold=50,
        minLineLength=20,
        maxLineGap=100
    )

    # average and extrapolate lines
    left_line, right_line = average_slope_intercept(frame, lines)
    averaged_lines = [left_line, right_line]

    # draw the averaged lines onto a blank image
    line_image = draw_lines(frame, averaged_lines)

    # overlay the lines onto the original frame
    final_image = cv.addWeighted(frame, 0.8, line_image, 1, 0)

    return final_image

frame_count = 0

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        processed_frame = process_frame(frame)
        out.write(processed_frame)

except Exception as e:
    print(f"error on processing: {e}")
finally:
    cap.release()
    out.release()
    print(f"video processed, output path: {output_video_path}")

video processed, output path: /content/drive/MyDrive/Colab Notebooks/Image-Processing-Project/highlighted_video-dashcam-night.mp4


In [6]:
from IPython.display import Video

# attempt to display the video inside colab
# does not work as intended because gdrive does not process the image fast enough
Video(output_video_path)